# **Import**

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

import pyro
import math
import pickle
import time
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

In [2]:
import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

## inputs set up

In [3]:
dimension_x = 4
# inner_dim = 6
dimension_y = inner_dim = 8

num_train_locations = 12
num_test_locations = 6

seed_train = 144
seed_test = 1301
variance = 3.2

X_train, X_test, Y_train, Y_test = Tools.generate_MVN_datasets(dimension_x, dimension_y, inner_dim, num_train_locations, num_test_locations, seed_train, seed_test, variance, 
                                                               sparse_density = 0.3, relationship = 'NInde')

## tensor

In [4]:
train_x = torch.tensor(X_train, dtype=torch.float32)

train_y = torch.tensor(Y_train, dtype=torch.float32)

test_x = torch.tensor(X_test, dtype=torch.float32)

test_y = torch.tensor(Y_test, dtype=torch.float32)

# **Models**

## local model

In [5]:
row_idx = 0

input_point = test_y[row_idx,:]
local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = 100)

bounds = bound.get_bounds(local_train_x)

### Local GP

In [5]:
def train_and_predict_LocalGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):

    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)
    
    LocalGP_models, LocalGP_likelihoods = Training.train_one_row_LocalGP(
        train_x, train_y, test_y, row_idx, covar_type = 'RBF', k_num=K_num, lr=0.05, num_iterations=5000, patience=10, device=Device
    )
    
    preds = Prediction.full_preds(LocalGP_models, LocalGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).cpu().detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)

    estimated_params, func_loss = Estimation.multi_start_estimation(LocalGP_models, LocalGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_Adam, 
                                                                num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return preds, estimated_params, func_loss

In [6]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_LocalGP)(row_idx, train_x, train_y, test_x, test_y) for row_idx in range(test_y.shape[0]))


In [7]:
full_test_preds_LocalGP = [item[0] for item in results]  
full_test_estimated_params_LocalGP = [item[1] for item in results]
full_test_estimated_params_LocalGP_loss = [item[2] for item in results]

# full_test_preds_LocalGP = np.vstack(results_preds)
# full_test_estimated_params_LocalGP = np.vstack(results_estimated_params)

MSE_LocalGP = np.mean((full_test_preds_LocalGP - test_y.numpy()) ** 2)
MSE_estimated_LocalGP = np.mean((full_test_estimated_params_LocalGP - test_x.numpy()) ** 2)

In [ ]:
np.array(full_test_estimated_params_LocalGP)

In [ ]:
test_x.numpy()

In [ ]:
print(MSE_LocalGP)
print(MSE_estimated_LocalGP)

In [ ]:
plt.boxplot(full_test_estimated_params_LocalGP_loss)

### Multi-task GP

In [6]:
def train_and_predict_MGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):


    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)

    MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_MultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], covar_type = 'RBF', 
                                                                                     lr=0.05, num_iterations=10000, patience=10, device=Device)

    # preds = Prediction.preds_for_one_model(MultitaskGP_models, MultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).squeeze().detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)


    estimated_params, _ = Estimation.multi_start_estimation(MultitaskGP_models, MultitaskGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_for_one_model_Adam, 
                                                                    num_starts=5, num_iterations=2000, lr=0.01, patience=20, 
                                                                    attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return estimated_params

In [7]:
data_sizes = [4096, 3576, 3076, 2548, 2048, 1524, 1024]

In [ ]:
train_y[:data_sizes[0],:]

In [ ]:
train_y

In [7]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_MGP)(row_idx, train_x, train_y, test_x, test_y) for row_idx in range(test_y.shape[0]))


In [ ]:
np.vstack(results)

In [ ]:
np.mean((np.vstack(results) - test_x.numpy()) ** 2)

In [7]:
full_test_preds_MGP = [item[0] for item in results]  
full_test_estimated_params_MGP = [item[1] for item in results]


MSE_MGP = np.mean((full_test_preds_MGP - test_y.numpy()) ** 2)
MSE_estimated_MGP = np.mean((full_test_estimated_params_MGP - test_x.numpy()) ** 2)

In [ ]:
print(MSE_MGP)
print(MSE_estimated_MGP)

In [ ]:
# full_test_estimated_params_MGP_loss = [item[2] for item in results]
plt.boxplot(full_test_estimated_params_MGP_loss)
plt.ylim(-0.001,0.021)

### DKL

#### NN + Local GP

In [5]:
Device = 'cpu'

In [27]:
def train_and_predict_NNLocalGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):

    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)
    
    NNLocalGP_models, NNLocalGP_likelihoods = Training.train_one_row_NNLocalGP_Parallel(train_x, train_y, test_y, row_idx, 
                                                                                        FeatureE.FeatureExtractor_6, 
                                                                                        covar_type = 'RBF', k_num = 100, lr=0.05, 
                                                                                        num_iterations=5000, patience=10, device=Device)
    
    preds = Prediction.full_preds(NNLocalGP_models, NNLocalGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).cpu().detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)

    estimated_params, func_loss = Estimation.multi_start_estimation(NNLocalGP_models, NNLocalGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_Adam, 
                                                                    num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                    attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return preds, estimated_params

In [28]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_NNLocalGP)(row_idx, train_x, train_y, test_x, test_y) for row_idx in range(test_y.shape[0]))


# results_preds = []
# results_estimated_params = []
# for row_idx in range(test_y.shape[0]):
#     preds, estimated_params = train_and_predict_NNLocalGP(row_idx, train_x, train_y, test_x, test_y)
#     results_preds.append(preds)
#     results_estimated_params.append(estimated_params)
#     # results.append((preds, estimated_params))

In [29]:
full_test_preds_NNLocalGP = [item[0] for item in results]  
full_test_estimated_params_NNLocalGP = [item[1] for item in results]


# full_test_preds_NNLocalGP = np.vstack(results_preds)
# full_test_estimated_params_NNLocalGP = np.vstack(results_estimated_params)

MSE_NNLocalGP = np.mean((full_test_preds_NNLocalGP - test_y.numpy()) ** 2)
MSE_estimated_NNLocalGP = np.mean((full_test_estimated_params_NNLocalGP - test_x.numpy()) ** 2)

In [ ]:
print(MSE_NNLocalGP)
print(MSE_estimated_NNLocalGP)

#### NN + Multi-task GP

In [ ]:
NNMultitaskGP_models, NNMultitaskGP_likelihoods = Training.train_one_row_NNMultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], 
                                                                                       feature_extractor_class = FeatureE.FeatureExtractor_1, lr=0.05, num_iterations=5000, patience=10, 
                                                                                       device='cpu')

In [13]:
def train_and_predict_NNMGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):


    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)

    NNMultitaskGP_models, NNMultitaskGP_likelihoods = Training.train_one_row_NNMultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], 
                                                                                           feature_extractor_class = FeatureE.FeatureExtractor_6, 
                                                                                           covar_type = 'Matern3/2', lr=0.025, num_iterations=5000, patience=20, device = Device)

    preds = Prediction.preds_for_one_model(NNMultitaskGP_models, NNMultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).squeeze().detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)


    estimated_params, func_loss = Estimation.multi_start_estimation(NNMultitaskGP_models, NNMultitaskGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_for_one_model_Adam, 
                                                                    num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                    attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return preds, estimated_params

In [14]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_NNMGP)(row_idx, train_x, train_y, test_x, test_y) for row_idx in range(test_y.shape[0]))


In [15]:
full_test_preds_NNMGP = [item[0] for item in results]  
full_test_estimated_params_NNMGP = [item[1] for item in results]


MSE_NNMGP = np.mean((full_test_preds_NNMGP - test_y.numpy()) ** 2)
MSE_estimated_NNMGP = np.mean((full_test_estimated_params_NNMGP - test_x.numpy()) ** 2)

In [ ]:
print(MSE_NNMGP)
print(MSE_estimated_NNMGP)

## Global model

In [5]:
Device = 'cpu'

### VGP

In [36]:
inducing_points = train_x[:500, :].to(Device)
VGP_models_500, VGP_likelihoods_500 = Training.train_full_VGP(train_x, train_y, inducing_points, lr=0.05, device=Device)

In [ ]:
# training_preds = Prediction.full_preds_for_VGP(VGP_models_500, VGP_likelihoods_500, train_x)
# print('MSE of Train data in VGP_models_200: {}'.format(torch.mean((training_preds - train_y) ** 2).item()))
test_preds = Prediction.full_preds_for_VGP(VGP_models_500, VGP_likelihoods_500, test_x)
print('MSE of Test data in VGP_models_200: {}'.format(torch.mean((test_preds - test_y) ** 2).item()))

In [38]:
def train_and_predict_VGP(VGP_models_500, VGP_likelihoods_500, row_idx, train_x, train_y, test_y, K_num = 100, Device = 'cpu'):

    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)
    

    estimated_params, func_loss = Estimation.multi_start_estimation(VGP_models_500, VGP_likelihoods_500, row_idx, test_y, bounds, Estimation.estimate_params_Adam_VGP, 
                                                                    num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                    attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return estimated_params

In [39]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_VGP)(VGP_models_500, VGP_likelihoods_500, row_idx, train_x, train_y, test_y) for row_idx in range(test_y.shape[0]))


In [ ]:
full_test_estimated_params_VGP = np.vstack(results)

MSE_estimated_VGP = np.mean((full_test_estimated_params_VGP - test_x.numpy()) ** 2)
print(MSE_estimated_VGP)

### Multi-task VGP

In [6]:
train_y_sub = train_y[:1024,:]

In [7]:
MVGP_models, MVGP_likelihoods = Training.train_full_MultitaskVGP(train_x, train_y_sub, 
                                                                 covar_type = 'RBF', num_latents = inner_dim + 1, num_inducing=100, 
                                                                 lr_hyper=0.05, lr_variational=0.1, num_iterations=5000, patience=20, device=Device)
# training_preds = Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, train_x)
# print('MSE of Train data in MVGP_models_100: {}'.format(torch.mean((training_preds - train_y) ** 2).item()))
test_preds = Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, test_x)
print('MSE of Test data in MVGP_models_100: {}'.format(torch.mean((test_preds - test_y) ** 2).item()))

RuntimeError: The size of tensor a (1024) must match the size of tensor b (4096) at non-singleton dimension 0

In [44]:
def train_and_predict_MVGP(MVGP_models, MVGP_likelihoods, row_idx, train_x, train_y, test_y, K_num = 100, Device = 'cpu'):

    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)
    

    estimated_params, func_loss = Estimation.multi_start_estimation(MVGP_models, MVGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_for_one_model_Adam, 
                                                                    num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                    attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return estimated_params

In [45]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_MVGP)(MVGP_models, MVGP_likelihoods, row_idx, train_x, train_y, test_y) for row_idx in range(test_y.shape[0]))


In [ ]:
full_test_estimated_params_MVGP = np.vstack(results)

MSE_estimated_MVGP = np.mean((full_test_estimated_params_MVGP - test_x.numpy()) ** 2)
print(MSE_estimated_MVGP)

### DGP

In [ ]:
DGP_2 = Training.train_full_DGP_2(train_x, train_y, num_hidden_dgp_dims = dimension_x, inducing_num = 100, num_iterations = 5000, patiences = 50, device=Device)

In [ ]:
full_test_mean = DGP_2.predict(test_x[0,:].unsqueeze(0))[0].detach().numpy()

for row_idx in range(1,test_y.shape[0]):
    test_mean_tmp = DGP_2.predict(test_x[row_idx,:].unsqueeze(0))[0].detach().numpy()
    full_test_mean = np.vstack((full_test_mean, test_mean_tmp))

print('MSE of Test data in DGP_2: {}'.format(np.mean((full_test_mean - test_y.detach().numpy()) ** 2)))

In [14]:
def train_and_predict_DGP(DGP_2, row_idx, train_x, train_y, test_y, K_num = 100, Device = 'cpu'):

    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)
    

    estimated_params, func_loss = Estimation.multi_start_estimation_nonliklihood(DGP_2, row_idx, test_y, bounds, Estimation.estimate_params_for_DGP_Adam, 
                                                                                 num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                                 attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return estimated_params

In [ ]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_DGP)(DGP_2, row_idx, train_x, train_y, test_y) for row_idx in range(test_y.shape[0]))


In [ ]:
full_test_estimated_params_DGP = np.vstack(results)

MSE_estimated_DGP = np.mean((full_test_estimated_params_DGP - test_x.numpy()) ** 2)
print(MSE_estimated_MVGP)

### NN

In [ ]:
NNmodel = Training.train_DNN_4(train_x, train_y, num_iterations = 10000, device=Device)

In [ ]:
# train_preds = Prediction.preds_for_DNN(NNmodel, train_x)
# Train_loss = torch.mean((train_preds - train_y) ** 2).item()
# print('Train_loss:', Train_loss)
test_preds  = Prediction.preds_for_DNN(NNmodel, test_x)
Test_loss = torch.mean((test_preds - test_y) ** 2).item()
print('Test_loss:', Test_loss)

In [8]:
def train_and_predict_NN(NNmodel, row_idx, train_x, train_y, test_y, K_num = 100, Device = 'cpu'):

    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)
    bounds = bound.get_bounds(local_train_x)
    

    estimated_params, func_loss = Estimation.multi_start_estimation_nonliklihood(NNmodel, row_idx, test_y, bounds, Estimation.estimate_params_for_NN_Adam, 
                                                                                 num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                                 attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

    return estimated_params

In [9]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_NN)(NNmodel, row_idx, train_x, train_y, test_y) for row_idx in range(test_y.shape[0]))


In [ ]:
full_test_estimated_params_NN = np.vstack(results)

MSE_estimated_NN = np.mean((full_test_estimated_params_NN - test_x.numpy()) ** 2)
print(MSE_estimated_NN)

# **Result**

In [ ]:
dimension_x_values = [5, 10, 15, 20, 30, 40]

k_num_values = [100, 200, 500, 1000]

dimension_y_values = [1, 2, 3]

inducing_points_values = [100, 200, 500, 1000]

In [ ]:
inner_dim_values = [2]
num_train_locations_values = [10,12]
dimension_x_values = [2, 4, 8]
k_num_values = [100, 200, 500, 1000]
dimension_y_values = [2, 4, 8]

In [ ]:
with open('ChangeDimofV/predictions_dict_LocalGP.pkl', 'rb') as file:
    predictions_dict_LocalGP = pickle.load(file)

In [ ]:
with open('ChangeDimofV/predictions_dict_VGP.pkl', 'rb') as file:
    predictions_dict_VGP = pickle.load(file)

## Local GP

In [ ]:
predictions_dict_LocalGP = pd.read_pickle("ChangeDimofV/predictions_dict_LocalGP.pkl")

In [ ]:
# Adjusting data structure for the updated visualization request
data_to_plot_means = {k_num: [] for k_num in k_num_values}

# Calculating means for each combination of dimension_x and k_num
for dimension_x in dimension_x_values:
    for k_num in k_num_values:
        # values = [predictions_dict_LocalGP[(dimension_x, k_num, y_val)] for y_val in dimension_y_values if (dimension_x, k_num, y_val) in predictions_dict_LocalGP]
        values = predictions_dict_LocalGP[(dimension_x, k_num, 1)]
        # concatenated_values = np.concatenate(values)
        data_to_plot_means[k_num].append(np.mean(values))

# Plotting
plt.figure(figsize=(12, 8))
colors = ['blue', 'green', 'red', 'purple']
for i, (k_num, means) in enumerate(data_to_plot_means.items()):
    plt.plot(dimension_x_values, means, 'o-', label=f'k={k_num}', color=colors[i])

plt.title('MSE of Predictions for Different K Numbers Across Dimension X Values')
plt.xlabel('Dimension X')
plt.ylabel('MSE of Predictions')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

# Looping over dimension_y_values to create three plots
for ax, dimension_y in zip(axs, dimension_y_values):
    data_to_plot_means = {k_num: [] for k_num in k_num_values}
    
    # Calculating means for each combination of dimension_x and k_num for the given dimension_y
    for dimension_x in dimension_x_values:
        for k_num in k_num_values:
            values = predictions_dict_LocalGP[(dimension_x, k_num, dimension_y)]
            data_to_plot_means[k_num].append(np.mean(values))
    
    # Plotting
    for i, (k_num, means) in enumerate(data_to_plot_means.items()):
        ax.plot(dimension_x_values, means, 'o-', label=f'k={k_num}', color=colors[i])
    
    ax.set_title(f'MSE for y={dimension_y}')
    ax.set_xlabel('Dimension X')
    if ax is axs[0]: # Only add y-label to the first subplot
        ax.set_ylabel('MSE of Predictions')
    ax.grid(True)

axs[0].legend(loc='upper left')
plt.suptitle('MSE of Predictions for Different K Numbers Across Dimension X Values')
plt.tight_layout()
plt.show()

In [ ]:
# Adjusting data structure for the updated visualization request
data_to_plot_means = {k_num: [] for k_num in k_num_values}
dimension_y_values = sorted(list(set(key[2] for key in predictions_dict_LocalGP.keys())))

# Calculating means for each combination of dimension_x and k_num
for dimension_y in dimension_y_values:
    for k_num in k_num_values:
        # values = [predictions_dict_LocalGP[(dimension_x, k_num, y_val)] for y_val in dimension_y_values if (dimension_x, k_num, y_val) in predictions_dict_LocalGP]
        values = predictions_dict_LocalGP[(20, k_num, dimension_y)]
        # concatenated_values = np.concatenate(values)
        data_to_plot_means[k_num].append(np.mean(values))

# Plotting
plt.figure(figsize=(12, 8))
colors = ['blue', 'green', 'red', 'purple']
for i, (k_num, means) in enumerate(data_to_plot_means.items()):
    plt.plot(dimension_y_values, means, 'o-', label=f'k={k_num}', color=colors[i])

plt.title('MSE of Predictions for Different K Numbers Across Dimension Y Values')
plt.xlabel('Dimension Y')
plt.xticks(dimension_y_values)
plt.ylabel('MSE of Predictions')
plt.legend()
plt.grid(True)
plt.show()


## NN+LocalGP

In [ ]:
predictions_dict_NNLocalGP = pd.read_pickle("ChangeDimofV/predictions_dict_NNLocalGP.pkl")
estimated_dict_NNLocalGP = pd.read_pickle("ChangeDimofV/estimated_dict_NNLocalGP.pkl")

In [ ]:
predictions_dict_NNLocalGP = pd.read_pickle("ChangeDimofV/predictions_dict_LocalGP_0.pkl")
estimated_dict_NNLocalGP = pd.read_pickle("ChangeDimofV/estimated_dict_LocalGP_0.pkl")

In [ ]:
for (inner_dim, num_train_locations, dimension_x, k_num, dimension_y), predictions in estimated_dict_NNLocalGP.items():
    print(f"Parameters: inner_dim = {inner_dim}, num_train_locations={num_train_locations}, dimension_x={dimension_x}, k_num={k_num}, dimension_y={dimension_y}")
    print(f"Predictions: {predictions}\n")

In [ ]:
for (inner_dim, num_train_locations, dimension_x, k_num, dimension_y), predictions in estimated_dict_NNLocalGP.items(): 
    average_prediction = sum(predictions) / len(predictions)

    print(f"Parameters: inner_dim = {inner_dim}, num_train_locations={num_train_locations}, dimension_x={dimension_x}, k_num={k_num}, dimension_y={dimension_y}")
    print(f"Average Prediction: {average_prediction}\n")

## VGP

In [ ]:
predictions_dict_VGP = pd.read_pickle("ChangeDimofV/predictions_dict_VGP.pkl")

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
colors = ['blue', 'green', 'red', 'purple']
# Looping over dimension_y_values to create three plots
for ax, dimension_y in zip(axs, dimension_y_values):
    data_to_plot_means = {inducing_num: [] for inducing_num in inducing_points_values}
    
    # Calculating means for each combination of dimension_x and k_num for the given dimension_y
    for dimension_x in dimension_x_values:
        for inducing_num in inducing_points_values:
            values = predictions_dict_VGP[(dimension_x, inducing_num, dimension_y)]
            data_to_plot_means[inducing_num].append(np.mean(values))
    
    # Plotting
    for i, (inducing_num, means) in enumerate(data_to_plot_means.items()):
        ax.plot(dimension_x_values, means, 'o-', label=f'inducing_num={inducing_num}', color=colors[i])
    
    ax.set_title(f'MSE for y={dimension_y}')
    ax.set_xlabel('Dimension X')
    if ax is axs[0]: # Only add y-label to the first subplot
        ax.set_ylabel('MSE of Predictions')
    ax.grid(True)

axs[0].legend(loc='upper left')
plt.yticks([0.02,0.05,0.11,0.12,0.14])
# ax.set_ylim([-0.00001, 0.12])
plt.suptitle('MSE of Predictions for Different Numbers of Inducing point Across Dimension X Values')
plt.tight_layout()
plt.show()